<a href="https://colab.research.google.com/github/ikanx101/G-Colab/blob/main/New_Items.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
#system("sudo apt-get install libglpk-dev")
#install.packages("ROI")
#install.packages("ompr")
#install.packages("ompr.roi")
#install.packages("ROI.plugin.glpk")

In [77]:
library(dplyr)
library(ompr)
library(ompr.roi)
library(ROI.plugin.glpk)
library(ggplot2)

rm(list=ls())

In [78]:
problem = 
data.frame(
  keterangan = c("Plant 1, jam","Plant 2, jam","Profit","Sales Potential"),
  item_1 = c(3,4,5,7),
  item_2 = c(4,6,7,5),
  item_3 = c(2,2,3,9),
  production_time_per_plant = c(30,40,NA,NA)
)

problem

keterangan,item_1,item_2,item_3,production_time_per_plant
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
"Plant 1, jam",3,4,2,30
"Plant 2, jam",4,6,2,40
Profit,5,7,3,NA
Sales Potential,7,5,9,NA


In [79]:
# hanya boleh pilih 1 plant saja dari 2 alternatif
# max hanya 2 produk saja yang bisa dipilih
# pertanyaannya: harus berapa konfigurasi produknya?

# i adalah item, ada 3 --> continuous
# j adalah plant, ada 2 --> binary

profit = c(5,7,3)
  # x[i] * profit[i] harus max

sales_pot = c(7,5,9)
  # x[i] <= sales_pot[i]

# waktu
  # 3*x[1] + 4*x[2] + 2*x[3] <= 30
  # 4*x[1] + 6*x[2] + 2*x[3] <= 40

# batas plant
  # y[j] <= 2


In [80]:
M = 10000
model = 
  MIPModel() %>%
  add_variable(x[i],
               i = 1:3,
               type = "continuous",
               lb = 0) %>%
  add_variable(y[j],
               j = 1:3,
               type = "binary") %>%
  add_variable(z[k],
               k = 1,
               type = "binary") %>%
  set_objective(sum_expr(x[i] * profit[i], i = 1:3),
                "max"
               ) %>%
  add_constraint(x[i] <= sales_pot[i], i = 1:3) %>%
  add_constraint(x[1] - y[1]*M <= 0) %>% 
  add_constraint(x[2] - y[2]*M <= 0) %>%
  add_constraint(x[3] - y[3]*M <= 0) %>%
  add_constraint(sum_expr(y[i], i=1:3) <= 2) %>%
  add_constraint(3*x[1] + 4*x[2] + 2*x[3] - M * z[1] <= 30 ) %>%
  add_constraint(4*x[1] + 6*x[2] + 2*x[3] + M * z[1] <= 40 + M)
model


Mixed integer linear optimization problem
Variables:
  Continuous: 3 
  Integer: 0 
  Binary: 5 
Model sense: maximize 
Constraints: 10 

In [81]:
result <- solve_model(model, with_ROI(solver = "glpk", verbose = TRUE))


<SOLVER MSG>  ----
GLPK Simplex Optimizer, v4.65
10 rows, 8 columns, 22 non-zeros
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (3)
*     7: obj =   9.700000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.65
10 rows, 8 columns, 22 non-zeros
5 integer variables, all of which are binary
Integer optimization begins...
Long-step dual simplex will be used
+     7: mip =     not found yet <=              +inf        (1; 0)
+    11: >>>>>   7.000000000e+01 <=   7.000000000e+01   0.0% (4; 0)
+    11: mip =   7.000000000e+01 <=     tree is empty   0.0% (0; 7)
INTEGER OPTIMAL SOLUTION FOUND
<!SOLVER MSG> ----


In [82]:
result$solution
problem

3*7 + 4*5
4*7 + 6*5

x[1] x[2] x[3] y[1] y[2] y[3] z[1] z[2] 
   7    5    0    1    1    0    1    0

keterangan,item_1,item_2,item_3,production_time_per_plant
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
"Plant 1, jam",3,4,2,30
"Plant 2, jam",4,6,2,40
Profit,5,7,3,NA
Sales Potential,7,5,9,NA


[1] 41

[1] 58